In [ ]:
!pip install requests openpyxl bs4
# 실행 전 필요라이브러리 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os, threading, json, re, time, datetime, pathlib
from pprint import pprint

import openpyxl, bs4, requests

save_path = "result"  # 파일이 저장될 폴더의 이름을 지정할 수 있다.
ses = requests.Session()

os.makedirs(save_path, exist_ok=True)

hdr = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
}


def get_books(word, page, delay=0, order="new", minlastup=None, maxlastup=None):
    """
    검색함수 : 해당 키워드로 검색 했을 때 X번째 페이지의 데이터들을 수집한다.
    :param word: 검색 키워드
    :param page: 해당 페이지
    :param delay: 함수 실행시 발생 하는 추가 딜레이
    :param order: 검색 타입
    :param minlastup: 검색 범위 시작 날짜
    :param maxlastup: 검색 범위 마지막 날짜
    :return:
    """
    url = "https://yomou.syosetu.com/search.php"
    payload = {
        'word': word,
        'order_former': 'search',
        'order': order,
        'notnizi': '1',
        'p': page,
    }

    if minlastup:
        payload['minlastup'] = minlastup
    if maxlastup:
        payload['maxlastup'] = maxlastup

    res = ses.get(url, params=payload, headers=hdr)
    soup = bs4.BeautifulSoup(res.content, "html.parser")
    books = soup.select("div.searchkekka_box")

    result = []
    for book in books:
        title_tag = book.select_one("div.novel_h > a.tl")
        title = title_tag.text.strip()
        contents_url = title_tag['href']

        state_tag = book.select_one("td.left")
        state = state_tag.text.strip()

        genre = ""
        keywords = []
        tmp_tags = book.select("td:nth-of-type(2) > a")
        for tag in tmp_tags:
            tag_href = tag['href']
            if "genre" in tag_href:
                genre = tag.text.strip()
            elif "word" in tag_href:
                keywords.append(tag.text.strip())

        book_content = book.text.strip()
        review_p = re.compile("レビュー数： (.*)件")
        s = review_p.search(book_content)
        review = 0
        if s:
            review_text = s.groups()[0]
            review_text = review_text.replace(" ", "").replace(",", "")
            review = int(review_text)

        recent_p = re.compile("最終更新日：(\d{4}/\d{2}/\d{2} \d{2}:\d{2})")
        s = recent_p.search(book_content)
        recent = None
        if s:
            recent_text = s.groups()[0]
            recent = datetime.datetime.strptime(recent_text, "%Y/%m/%d %H:%M")

        item = {
            'title': title,
            'state': state,
            'genre': genre,
            'keywords': keywords,
            'review': review,
            'recent': recent,
            'url': contents_url,
        }

        result.append(item)

    time.sleep(delay)

    return result


def get_contents(url, delay=0):
    """
    선택된 소설의 세부 에피소드들의 정보를 수집한다.
    :param url: 선택된 소설의 url
    :param delay: 추가 딜레이
    :return: 
    """
    res = ses.get(url, headers=hdr)
    soup = bs4.BeautifulSoup(res.content, "html.parser")
    contents_list = soup.select("dl.novel_sublist2")
    contents = []
    for cont in contents_list:
        title_tag = cont.select_one("dd > a")
        title = title_tag.text.strip()
        cont_url = title_tag['href']

        update_tag = cont.select_one("dt")
        modify_tag = update_tag.select_one("span")
        if modify_tag:
            modify_tag.decompose()
        update_text = update_tag.text.strip()
        update = datetime.datetime.strptime(update_text, "%Y/%m/%d %H:%M")

        item = {
            'title': title,
            'update': update,
            'url': f"https://ncode.syosetu.com{cont_url}",
        }
        contents.append(item)

    time.sleep(delay)

    return contents


def make_excel(genres, keywords, months, subtitle=None):
    """
    검색된 데이터들을 바탕으로 엑셀파일을 생성한다.
    :param genres:
    :param keywords:
    :param months:
    :param subtitle: 엑셀파일 결과물의 이름에 반영할 부제목
    :return:
    """
    wb = openpyxl.Workbook()
    genre_sht = wb.create_sheet('장르', 0)
    keyword_sht = wb.create_sheet('키워드', 1)
    month_sht = wb.create_sheet('연도별 월별 작품수', 2)

    genre_sht.append(["장르", "소설 수"])
    keyword_sht.append(["키워드", "소설 수"])
    month_sht.append(["날짜", "소설 수"])

    for (g, count) in sorted(list(genres.items()), key=lambda x: x[1], reverse=True):
        genre_sht.append([g, count])

    for (k, count) in sorted(list(keywords.items()), key=lambda x: x[1], reverse=True):
        keyword_sht.append([k, count])

    for (m, count) in sorted(list(months.items()), key=lambda x: x[0]):
        month_sht.append([m, count])

    dstring = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

    filepath = f"{save_path}/결과_{dstring}.xlsx"
    if subtitle:
        filepath = f"result/결과_{subtitle}_{dstring}.xlsx"
    wb.save(filepath)

    print(f"엑셀 파일 생성 완료 : {filepath}")


def process(word, number_of_pages, delay, order="new", subtitle=None, minlastup=None, maxlastup=None):
    """
    전체적인 프로세스를 진행한다.
    1. 검색 키워드로 소설 리스트 수집
    2. 리스트에서 나온 소설 url을 이용하여 소설 에피소드들의 데이터를 수집
    3. 1번과 2번 과전 반복
    4. 엑셀파일 생성
    """
    
    genres = dict()
    months = dict()
    keywords = dict()

    for page in range(1, number_of_pages + 1):
        books = get_books(word, page, order=order, minlastup=minlastup, maxlastup=maxlastup)
        for n, book in enumerate(books[:]):
            contents_url = book['url']
            contents = get_contents(contents_url, delay=delay)
            print(f"[ {(page - 1) * 20 + n + 1:3} ][ contents : {len(contents):3} ] : {book}")
            bkeywords = book['keywords']
            bgenre = book['genre']

            for bk in bkeywords:
                if not bk in keywords:
                    keywords[bk] = 1
                else:
                    keywords[bk] += 1

            if not bgenre in genres:
                genres[bgenre] = 1
            else:
                genres[bgenre] += 1

            for content in contents[:1]:
                date = content['update'].strftime("%Y-%m")
                if not date in months:
                    months[date] = 1
                else:
                    months[date] += 1

    make_excel(genres, keywords, months, subtitle=subtitle)


In [ ]:
"""
pages 와 delay를 조절하여 테스트 진행
pages는 해당 키워드로 검색하여 찾는 페이지의 수를 지정
하나의 페이지에 20개의 소설이 검색되니 2000개의 소설에 대해서 검색을 진행하려면 100페이지로 설정
딜레이는 기본적으로 1을 주고 진행
"""
pages = 1
delay = 1
process("悪役令嬢", pages, delay)

In [ ]:
    """
    종합포인트순으로 나열
    """
pages = 1
delay = 1
process("悪役令嬢", pages, delay, order="hyoka", subtitle="종합포인트")

In [ ]:
    """
    투고날짜 순으로 나열
    """
pages = 1
delay = 1
process("悪役令嬢", pages, delay, subtitle="초창기", minlastup="2013/03/01", maxlastup="2017/01/01")